# GBQ

In [1]:
from packages.dev_thesis.gbq_utils import GBQUtils
import pandas as pd
import json
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
gbq = GBQUtils()
client = gbq.get_client()

In [4]:
table_id = "danish-endurance-analytics.nl2sql.amazon_orders"

## Get long Context Window

In [5]:
distinct_values = gbq.get_long_context_window_fields(table_id=table_id, fetch_all=['market','product_marketing_category', 'product_type', 'product_pack', 'product_category', 'product_size', 'product_colour'], fetch_partial=['order_id', 'purchase_date', 'buyer_email', 'child_asin', 'e_conomic_number', 'product_name','product_and_pack', 'gross_sales', 'units_sold'])

In [10]:
import json 
with open('full_context_metada.json', 'w') as f:
    json.dump(distinct_values, f, indent=4)

In [11]:
# Writing JSON data to a text file
with open('full_context_metada.txt', 'w') as text_file:
    text_file.write(json.dumps(distinct_values, indent=4))

## Get OpenAI table Schema

In [5]:
table_id = "danish-endurance-analytics.nl2sql.amazon_orders"
print(gbq.get_openAI_table_schema(table_id=table_id))

danish-endurance-analytics.nl2sql.amazon_orders('order_id', 'purchase_date', 'buyer_email', 'market', 'child_asin', 'e_conomic_number', 'product_marketing_category', 'product_name', 'product_pack', 'product_and_pack', 'product_category', 'product_type', 'product_size', 'product_colour', 'gross_sales', 'units_sold')


In [6]:
column_distinct = gbq.get_distinct_values(table_id=table_id, fetch_all=['market','product_marketing_category', 'product_type', 'product_pack', 'product_category', 'product_size', 'product_colour'], fetch_partial=['order_id', 'purchase_date', 'buyer_email', 'child_asin', 'e_conomic_number', 'product_name','product_and_pack', 'gross_sales', 'units_sold'])
column_distinct

{'table_id': 'danish-endurance-analytics.nl2sql.amazon_orders',
 'table_description': 'The table contains information about the sales ',
 'fields': [{'field_name': 'order_id',
   'field_description': 'The unique identifier for each order.',
   'field_dataType': 'STRING',
   'distinct_values': '171-4074050-5185965, 403-2634040-4720338, 407-2338784-2005960, 408-0554643-1967553, 402-7148899-6689942, 406-7327640-2754734, 402-7446542-7092330, 407-7693202-9421956, 405-2934473-8325929, 404-3653894-6259543, 403-2546701-8683524, 408-5254396-3122728, 407-6821087-6205951, 406-3850293-1427513, 408-6741415-9600338, 402-9522861-5399513, 407-1850680-8336332, 405-7798204-5405154, 408-2263243-2937127, 404-7917494-9923506, '},
  {'field_name': 'purchase_date',
   'field_description': 'The date when the purchase was made.',
   'field_dataType': 'DATE',
   'distinct_values': '2023-03-01, 2023-09-04, 2023-05-05, 2023-06-05, 2023-12-27, 2023-12-06, 2021-03-22, 2022-11-21, 2020-12-06, 2022-11-24, 2022-12-14,

In [11]:
# Step 2 and 3: Convert dictionary to JSON and save to a file
with open('data.json', 'w') as json_file:
    json.dump(column_distinct, json_file, indent=4)

## Generate Field Values LookUp

In [1]:
from packages.dev_thesis.gbq_utils import GBQUtils

gbq = GBQUtils()
table_id = "danish-endurance-analytics.nl2sql.amazon_orders"
client = gbq.get_client()
print(gbq.get_openAI_table_schema(table_id=table_id))


danish-endurance-analytics.nl2sql.amazon_orders('order_id', 'purchase_date', 'buyer_email', 'market_alpha_2_code', 'child_asin', 'e_conomic_number', 'product_marketing_category', 'product_name', 'product_pack', 'product_and_pack', 'product_category', 'product_type', 'product_size', 'product_colour', 'gross_sales', 'units_sold')


In [2]:
column_distinct = gbq.get_and_save_field_values_lookup(table_id=table_id, fetch_all=['market_alpha_2_code','product_marketing_category', 'e_conomic_number', 'product_name', 'product_pack','product_and_pack',  'product_category', 'product_type', 'product_size', 'product_colour', 'child_asin'], fetch_partial=[])
column_distinct

{'table_id': 'danish-endurance-analytics.nl2sql.amazon_orders',
 'table_description': 'The table contains information about the sales ',
 'fields': [{'field_name': 'market_alpha_2_code',
   'field_description': 'No description available',
   'field_dataType': 'STRING',
   'distinct_values': ['PL',
    'JP',
    'ES',
    'GB',
    'TR',
    'CA',
    'SE',
    'FR',
    'NL',
    'US',
    'BE',
    'DE',
    'IT',
    'MX']},
  {'field_name': 'child_asin',
   'field_description': 'The unique identifier for the specific product variant purchased; can be used interchangeably with economic number',
   'field_dataType': 'STRING',
   'distinct_values': ['B07QB3DS7F',
    'B07THB3C5M',
    'B07QB1X5R3',
    'B01EYLF7SU',
    'B01FK5A8LU',
    'B01FK5A8EW',
    'B01FK5A94Q',
    'B082V9WMND',
    'B08DP4LY1F',
    'B06XPKDVHH',
    'B0771RP7JB',
    'B07D8WZWHH',
    'B09WYX39SG',
    'B08DY96QSR',
    'B082KZTYYJ',
    'B07HKB1RS2',
    'B07GDJ9B8Y',
    'B07GDJQLRL',
    'B083V229X9',
    

# Archive

## Query on GBQ example

In [ ]:
# Define SQL query
sql_query = """
SELECT
    distinct product_marketing_category 
FROM {}
""".format(table_id)

# Perform SQL query and get results
query_job = client.query(sql_query)
results = query_job.result()

results_list = results.to_dataframe().iloc[:, 0].values.tolist()

# At this point, result_string contains all the values from the queried column as a string
print(results_list)


['Fitness Accessories', 'Female Underwear', 'Casual Wear', 'Compression Socks', 'Hiking Socks', 'Dress Socks', 'Male Underwear', 'Headwear', 'Baselayer', 'Sports Socks', 'Running Wear', 'Cycling Wear', 'Outdoor Wear']


In [ ]:
import pandas as pd
from datetime import datetime

def analyze_date_columns(df):
    # Initialize an empty string to store the results
    result = ""
    
    # Identify date columns by trying to convert them and catching conversion failures
    date_columns = []
    for col in df.columns:
        if df[col].dtype == 'datetime64[ns]':
            try:
                pd.to_datetime(df[col])
                date_columns.append(col)
            except ValueError:
                continue
    
    # For each date column, perform the analysis
    for col in date_columns:
        df[col] = pd.to_datetime(df[col])  # Ensure conversion to datetime
        unique_dates = df[col].nunique()
        most_common_date = df[col].mode()[0]
        most_common_date_freq = df[col].value_counts().iloc[0]
        
        # Calculate skewness proxy
        median_date = df[col].median()
        before_median = (df[col] < median_date).sum()
        after_median = (df[col] > median_date).sum()
        skewness_proxy = (after_median - before_median) / df[col].notnull().sum()
        
        # Append the analysis to the result string
        result += (f"Column: {col}\n"
                   f"Total Unique Dates: {unique_dates}\n"
                   f"Most Common Date: {most_common_date.date()}, Frequency: {most_common_date_freq}\n"
                   f"Skewness Proxy (Positive indicates skew towards later dates, Negative towards earlier dates): {skewness_proxy:.2f}\n"
                   "-----\n")
    
    if not date_columns:
        result = "No columns with dbdate format were identified."
    
    return result[:1000]  # Ensure the result string does not exceed 1000 words

# Example usage:
# df = pd.read_csv('your_dataset.csv')
# print(analyze_date_columns(df))